In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")


LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


### Runnable
* `RunnablePassthrough` 는 입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있습니다.
* `RunnablePassthrough()` 가 단독으로 호출되면, 단순히 입력을 받아 그대로 전달합니다.
* `RunnablePassthrough.assign(...)` 방식으로 호출되면, 입력을 받아 assign 함수에 전달된 추가 인수를 추가합니다.

In [4]:
# RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
# define llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
) 
prompt = PromptTemplate.from_template("{num} 의 10배는?")

In [11]:
# Create chain
chain = prompt | llm

chain.invoke({"num": 10})

AIMessage(content='10의 10배는 100입니다. (10 x 10 = 100)', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--7125f29a-aefd-42b6-84f3-f66875d554b5-0', usage_metadata={'input_tokens': 9, 'output_tokens': 28, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}})

In [12]:
# execute invoke another method
chain.invoke(15)

AIMessage(content='15의 10배는 15 x 10 = 150 입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--8e8ff8d6-3387-4420-a8d9-b250540013c7-0', usage_metadata={'input_tokens': 9, 'output_tokens': 23, 'total_tokens': 32, 'input_token_details': {'cache_read': 0}})

In [13]:
from langchain_core.runnables import RunnablePassthrough

# runnable
RunnablePassthrough().invoke({"num":10})

{'num': 10}

In [17]:
# RunnablePassthrough.assign
# lambda 및 함수를 활용해서 input 값을 할당할 수 있음
# 입력 키: num, 할당(assign) 키: new_num
def custom_func(input_dict):
    # 복잡한 로직을 포함할 수 있는 커스텀 함수
    res = input_dict['num'] * 20
    return res
(RunnablePassthrough.assign(new_num=lambda x: custom_func(x))).invoke({"num": 1})

{'num': 1, 'new_num': 20}

In [18]:
# RunnableParallel
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스를 생성합니다. 이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행할 수 있습니다.
runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달합니다. 이는 입력된 데이터를 그대로 통과시키는 역할을 합니다.
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용하여, 'mult' 람다 함수를 할당합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가시킵니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더합니다.
    modified=lambda x: x["num"] + 1,
)

# runnable 인스턴스에 {'num': 1} 딕셔너리를 입력으로 전달하여 invoke 메소드를 호출합니다.
runnable.invoke({"num": 1})


{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [19]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | llm
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | llm
)


In [20]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")


{'capital': AIMessage(content='대한민국의 수도는 서울입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--08b8c535-6a99-41c7-9b50-89420a5a2033-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'cache_read': 0}}),
 'area': AIMessage(content='대한민국의 면적은 약 100,364 제곱킬로미터 (km²)입니다. 이는 한반도 전체 면적의 약 45%에 해당합니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--b10371bc-d557-40d5-ae92-55ad7b369405-0', usage_metadata={'input_tokens': 9, 'output_tokens': 48, 'total_tokens': 57, 'input_token_details': {'cache_read': 0}})}

In [ ]:
# runnablelambda
# custom 함수를 맴핑할 수 있음
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from datetime import datetime


def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)


'Aug-19'

In [6]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [10]:
print(chain.invoke(5))

물론입니다. 8월 19일에 태어난 유명인 5명을 소개합니다.

*   **빌 클린턴** - 1946년 8월 19일
*   **존 스토모스** - 1963년 8월 19일
*   **매튜 페리** - 1969년 8월 19일
*   **케빈 내쉬** - 1959년 8월 19일
*   **다이애나 무어** - 1945년 8월 19일


In [11]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda


# 문장의 길이를 반환하는 함수입니다.
def length_function(text):
    return len(text)


# 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")

chain1 = prompt | llm

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | llm
)


In [12]:
chain.invoke({
    "word1" : "hello",
    "word2" : "python312"
})

AIMessage(content='5 + 45 = 50 입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f1449da4-570d-4b7a-a9d1-062fd1c975d1-0', usage_metadata={'input_tokens': 12, 'output_tokens': 12, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})